In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import date
from datetime import time
from datetime import timedelta
import datetime

In [ ]:
# Preprocess current sp500 and sp500 component changes
# wiki_sp = current sp500
# wiki_change = wiki sp500 component changes
# sp_historic = sp500 component list since 2011
def read_and_preprocess(url, sp_historic_path):
    table1 = pd.read_html(url)
    wiki_sp = table1[0]
    wiki_sp = wiki_sp[['Symbol']]

    table2 = pd.read_html(url, header=0)
    wiki_change = table1[1]
    
    sp_historic = pd.read_csv(sp_historic_path, sep='|')
    sp_historic['date'] = pd.to_datetime(sp_historic['date'])
    sp_historic['tickers'] = sp_historic['tickers'].apply(lambda x: sorted(x.split(',')))
    
    return wiki_sp, wiki_change, sp_historic


def wiki_change_formatting(wiki_change):
    # Drop extra header and unwanted cols
    wiki_change = wiki_change.reset_index(drop=True)
    wiki_change = wiki_change.drop(['Reason', 'Security'], axis=1, level=1)
    bottom_headers = wiki_change.columns.get_level_values(0)
    top_headers = wiki_change.columns.get_level_values(1)
    wiki_change.columns = bottom_headers
    # Rename Cols
    wiki_change = wiki_change.rename(columns={'Date': 'date', 'Added': 'added', 'Removed': 'removed'})
    # Fix date format
    wiki_change['date'] = pd.to_datetime(wiki_change['date'])
    wiki_change['date'] = wiki_change['date'].dt.strftime('%Y-%m-%d')
    wiki_change['date'] = pd.to_datetime(wiki_change['date'])
    # Filter off dates before current date
    wiki_change = wiki_change.loc[wiki_change['date'] >= pd.to_datetime('2023-06-21')]
    # Replace NaN values with None in 'Added' and 'Removed' columns
    wiki_change['added'] = wiki_change['added'].fillna('None')
    wiki_change['removed'] = wiki_change['removed'].fillna('None')
    # Group by 'Date' column and combine 'Added' and 'Removed' values as a list
    wiki_change = wiki_change.groupby('date').agg(lambda x: x.tolist()).reset_index()
    # Sort by date
    wiki_change = wiki_change.sort_values(by='date', ascending=True)
    wiki_change = wiki_change.reset_index(drop=True)
    
    return wiki_change

# Update sp_historic with changes from wiki_change    
def historic_data_process(wiki_change, sp_historic):
    for change in wiki_change.itertuples():
        # Assign values
        date = change.date
        added_tickers = change.added
        removed_tickers = change.removed

        new_row = sp_historic.tail(1)
        tickers = list(new_row['tickers'].values[0])

        # Skip over 'None' values and proceed to next valid symbol in added_tickers
        index = 0
        while index < len(added_tickers):
            ticker = added_tickers[index]
            if ticker == 'None':
                index += 1
                continue
            tickers.append(ticker)
            index += 1

        # Add or remove tickers in list    
        tickers = list(set(tickers) - set(removed_tickers))
        tickers = sorted(tickers)

        # Store updates & concat
        d = {'date': date, 'tickers': [tickers]}
        new_entry = pd.DataFrame(d)
        sp_historic = pd.concat([sp_historic, new_entry]).reset_index(drop=True)
        
    # Define ticker replacements
    ticker_replacements = {
    'PKI': 'RVTY',
    'FISV': 'FI',
    'RE': 'EG'
    }
    # Replace tickers with updated tickers
    sp_historic['tickers'] = sp_historic['tickers'].apply(lambda x: sorted([ticker_replacements.get(ticker, ticker) for ticker in x]))
    
    # Formatting
    # Fix date format
    sp_historic['date'] = pd.to_datetime(sp_historic['date'])
    sp_historic['date'] = sp_historic['date'].dt.strftime('%Y-%m-%d')
    sp_historic['date'] = pd.to_datetime(sp_historic['date'])
    sp_historic = sp_historic.sort_values(by='date')
    # Resample with daily frequency
    # Set 'date' column as the index
    #sp_historic = sp_historic.set_index('date')
    # Resample the dataframe with daily frequency and forward fill the missing values
    #sp_historic = sp_historic.resample('D').ffill()
    # Reset the index and restore 'date' as a column
    #sp_historic = sp_historic.reset_index()
    #sp_historic = sp_historic.sort_values('date')
    
    return sp_historic

# Update sp_historic with corp actions
def process_corporate_actions(sp_historic, s_changes_path):
    # Update sp historic with corporate action ticker changes
    s_changes = pd.read_csv(s_changes_path, sep='|')
    s_changes = s_changes.sort_values(by='date', ascending=False)
    s_changes = s_changes[s_changes['date'] <= '2023-06-20']
    s_changes = s_changes.reset_index(drop=True)

    # Group changes by day
    grouped_changes = s_changes.groupby('date')

    # Iterate over each day going back in time
    for date, group in reversed(list(grouped_changes)):

        # Get tickers for the day before the change date
        prev_date = pd.to_datetime(date) - pd.DateOffset(days=1)
        prev_day_tickers = sp_historic.loc[sp_historic['date'] == prev_date, 'tickers']
        if prev_day_tickers.empty:
            break
        prev_day_tickers = prev_day_tickers.iloc[0]

        removed_tickers = []
        added_tickers = []
        for index, row in group.iterrows():
            symbol = row['symbol']
            prev_symbol = row['prev_symbol']
            if symbol in prev_day_tickers:
                removed_tickers.append(symbol)
                added_tickers.append(prev_symbol)

        removed_tickers = set(removed_tickers)
        added_tickers = set(added_tickers)
    
        # Find tickers only added or removed
        unique_added = added_tickers - removed_tickers
        unique_removed = removed_tickers - added_tickers

        def corporate_action(row, unique_added, unique_removed, prev_day_tickers):
            tickers = row
            updated_tickers = list(set(tickers) - set(unique_removed))
            updated_tickers += unique_added
            return sorted(updated_tickers)
            
        # Update tickers in sp_historic for current day
        sp_historic.loc[sp_historic['date'] < date, 'tickers'] = sp_historic.loc[sp_historic['date'] < date, 'tickers'].apply(lambda x: corporate_action(x, unique_added, unique_removed, prev_day_tickers))

    # Sort sp_historic by date
    sp_historic = sp_historic.sort_values(by='date').reset_index(drop=True)
    
    return sp_historic


def main(url, sp_historical_path, s_changes_path):
    wiki_sp, wiki_change, sp_historic = read_and_preprocess(url, sp_historic_path)
    wiki_change = wiki_change_formatting(wiki_change)
    sp_historic = historic_data_process(wiki_change, sp_historic)
    
    # Update with previous tickers if no changes from wiki sp
    # Get today's date
    today = pd.to_datetime('today').normalize()

    # If the latest date in the data is before today, append today's data
    if sp_historic['date'].iloc[-1] < today:
        last_tickers = sp_historic['tickers'].iloc[-1]
        new_row = {'date': today, 'tickers': last_tickers}
        sp_historic = sp_historic.append(new_row, ignore_index=True)
    
    sp_historic = process_corporate_actions(sp_historic, s_changes_path)
    
    return sp_historic

if __name__ == "__main__":
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_historic_path = '/home/rdoss/list_fixes/sp500_fix/output_check'
    s_changes_path = '/home/rdoss/list_fixes/sp500_fix/ticker_changes.csv'
    sp_historic = main(url, sp_historic_path, s_changes_path)
    
    print(sp_historic)
